#1. 구글드라이브와 연동


In [ ]:
##colab내 존재하는 content폴더에 drive폴더를 새로 만들고 거기에 개인 구글드라이브 내용을 넣는다 
 
from google.colab import drive
drive.mount('/content/drive')       

Mounted at /content/drive


#2. StyleGan2-ADA 초기 환경 세팅

- StyleGan2-ADA는 텐서플로우 Tensorflow 1에서만 작동한다.
- GPU환경 필요 : - colab에서 실행시 런타임 유형에서 gpu환경설정
- opensimplex 설치 필요

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [ ]:
##Tensorflow 1로 버전 설정

%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
##GPU환경 확인

!nvidia-smi

Fri Jun 11 03:28:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
##opensimplex 설치

!pip install opensimplex

#3. Generating COVID19 X-ray image

In [ ]:
##generate.py가 담긴 폴더로 이동

%cd ..
%cd content/drive/MyDrive/colab-sg2-ada/stylegan2-ada

/
/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada


In [ ]:
##훈련된 모델(.pkl파일)을 generate.py를 실행시키며 인자로 넣어줌
##seeds : 생성할 이미지의 개수를 정하는 부분, 여기서는 100개의 이미지를 생성한다
##outdir : 생성된 이미지가 저장될 디렉토리를 지정한다

!python generate.py generate-images --network='/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00000-COVID19-mirror-paper256-bg-resumeffhq256/network-snapshot-000221.pkl' --seeds=0-99 --outdir='./out_101'
#!python generate.py generate-images --network='/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00006-COVID_19-mirror-paper256-bg-resumecustom/network-snapshot-000655.pkl' --seeds=0-99 --outdir='./out_100'

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3637248/45929032 bytes (7.9%)7733248/45929032 bytes (16.8%)11829248/45929032 bytes (25.8%)15851520/45929032 bytes (34.5%)20054016/45929032 bytes (43.7%)24158208/45929032 bytes (52.6%)28409856/45929032 bytes (61.9%)32358400/45929032 bytes (70.5%)36618240/45929032 bytes (79.7%)40353792/45929032 bytes (87.9%)44400640/45929032 bytes (96.7%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

#4. 생성된 이미지를 질병분류모델로 분류하기
- 위에서는 COVID19이미지를 100개를 생성했음. 
- 따라서 분류결과 COVID19로 판단된 이미지가 많을수록 학습된 StyleGAN2-ADA모델의 성능이 좋은것이라 판단할 수 있다.

In [ ]:
##라이브러리 불러오기

import numpy as np 
import cv2
import os
from keras.models import load_model

In [ ]:
##생성된 가상 X-ray이미지 가공 - 1. 이미지 가공 함수 만들기

label = 'generated_img'
img_size = 244

def get_generated_data(data_dir):
    data = []  
    path = os.path.join(data_dir)    # data_dir = /content/chest_xray/chest_xray/train, label = 'PNEUMONIA' 이라면 os.path.join(data_dir, label)의 결과는 "/content/chest_xray/chest_xray/train/PNEUMONIA" 이다
    for img in os.listdir(path):
        try:
            img_arr = cv2.imread(os.path.join(path, img)) # 로컬경로의 이미지 파일 읽는법, GRAYSCALE : 이미지를 흑백으로 읽어들임
            resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
            data.append(resized_arr)
        except Exception as e:
            print(e)
    return data

In [ ]:
##생성된 가상 X-ray이미지 가공 - 2. 이미지 객체 저장

# generated_img = get_generated_data('/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/out_2')
generated_img = get_generated_data('/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/out_101')

In [ ]:
##생성된 가상 X-ray이미지 가공 - 3. 넘파이 어레이화

generated_img = np.array(generated_img)

In [ ]:
##생성된 가상 X-ray이미지 가공 - 4. 3D Vector 변환

width, height, channel = 244, 244, 3 # 이미지 사이즈 244*244 pixel 
generated_img = generated_img.reshape((generated_img.shape[0], width, height, channel))

print(generated_img.shape)

(100, 244, 244, 3)


In [ ]:
##질병분류모델 불러오기
##에러메세지가 뜨면 다음 셀의 !pip install h5py==2.10.0를 실행시키고
##런타임 재시작 후 "4. 생성된 이미지를 질병분류모델로 분류하기" 처음서부터 다시 실행

loaded_model = load_model('/content/drive/MyDrive/코드스테이츠/개인프로젝트/section6/Xray_classification_model_first.h5')

In [ ]:
##코랩에서는 반드시 버전을 다음과 같이 다운그레이드 해준 후 restart the runtime을 해준다
!pip install h5py==2.10.0

     |████████████████████████████████| 2.9MB 13.0MB/s 
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [ ]:
##질병분류모델 작동(COVID_19의 lable : 1)

new_predictions = loaded_model.predict_classes(generated_img)
new_predictions = new_predictions.reshape(1,-1)[0]
new_predictions   #예측결과

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1,
       1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
##예측결과인 넘파이배열을 파이썬리스트화

new_predictions = new_predictions.tolist()
new_predictions

[1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [ ]:
##StyleGan2-ADA로 생성한 이미지 중 질병분류모델에 의해 COVID19 이미지로 판명된 이미지 수

print(f"COVID19 이미지는, {len(new_predictions)}개의 이미지중 {new_predictions.count(1)}개 있습니다.")

COVID19 이미지는, 100개의 이미지중 84개 있습니다.


In [ ]:
## 질병분류모델의 정확성을 고려한 신뢰도

print(f"질병 분류모델에 의해 COVID19로 분류된 이미지 {new_predictions.count(1)}개 중 {round(new_predictions.count(1) * 0.9591)}개만큼의 신뢰도가 있습니다")

질병 분류모델에 의해 COVID19로 분류된 이미지 84개 중 81개만큼의 신뢰도가 있습니다
